In [1]:
import requests
from bs4 import BeautifulSoup as bsp
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sqlite3

import json
import time
from datetime import datetime, timedelta
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import pytesseract
from PIL import Image
from urllib import request

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

In [3]:
# DB에 와디즈의 게시물 제목과 내용 DB에 담기(연습으로 3개 게시물)
driver = Chrome()
driver.get("https://www.wadiz.kr/web/wreward/category/288?keyword=&endYn=ALL&order=recommend") 
driver.maximize_window()

In [4]:
#스크롤다운
body = driver.find_element_by_tag_name('body')#스크롤하기 위해 소스 추출
i = 0
while i <= 1:
    body.send_keys(Keys.PAGE_DOWN)
    print("count : ", i)
    i = i + 1

count :  0
count :  1


In [5]:
hrefs=[]
table =driver.find_elements_by_class_name("CommonCard_image__vaqkf")
# # table =driver.find_element_by_css_selector("CardLink_link__1k83H CommonCard_image__vaqkf")
# elems = table.find_elements_by_xpath("//a[@href]")
for i in table:
    hrefs.append(i.get_attribute('href'))

In [6]:
# PYTHON으로 SQLITE3의 DB에 데이터 담기
dbname='wadiz48.db'
def insData(d, tblname='wadiztable_prac'):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        sql='select nkey from '+tblname+' where nkey="'+d[-1]+'"'
        ckkey=cur.execute(sql).fetchall()
        if ckkey:
            print('-',end='')
        
        else:
            insql='insert into '+tblname+' (ntitle,ncontent,npercent,nsup,nkey) values (?,?,?,?,?)'
            cur.execute(insql,d)
            print('.',end='')

In [7]:
for i  in hrefs:
    url = i
    webpage = requests.get(url)
    soup = bsp(webpage.content, "html.parser")

    # ntitle
    title = soup.find_all('a')
    ntitle = title[4].contents
    ntitle

    #ncontent
    reqheader = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0;Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'} 
    src=requests.get(url,headers=reqheader)
    cont=src.content.strip()#.decode('cp949') # 디코드 하고 가져오기
    html=bsp(cont,'html.parser')
    tgts=html.select('.inner-contents p')

    ncontent_list = []
    for tgt in tgts:
        ncontent_list.append(tgt.text)
    ncontent = " ".join(ncontent_list)

    #npercent
    percent = soup.find_all('p', 'achievement-rate')
    npercent = percent[0].contents[0].contents

    #nsup
    sup = soup.find_all('p', 'total-supporter')
    nsup = sup[0].contents[0].contents

    #nkey
    nkey = url.split('/')[6]
    
    print(ntitle, npercent, nsup, nkey)
    
    #insert_wadiz(ntitle, ncontent, npercent, nsup, nkey)

['근육통? 참지 마세요! 작지만 강한 근육통 완화 패치 [패인팔팔패스]'] ['2466'] ['386'] 111218?acid=10009857&_refer_section_st=REWARD_0
['한국인의 발과 한국 산에 찰떡 맞춤! K-등산화 #사나래고어텍스 [네파]'] ['6093'] ['562'] 120950
['[핸드폰 자석클립 거치대] 내 몸에 핸드폰을 붙인다!'] ['575'] ['168'] 117114
['[400개한정] 어? 귀엽다~ 진로XRETOO 보고 싶어도 조금만 참아요'] ['307'] ['35'] 117176
['데님이 시원해?ㅣ2WAY핏, 10가지 장점! 텐셀100% 스마트 텐셀데님'] ['282'] ['44'] 104051
['[소소한관심] 멸종위기 토종돌고래 "상괭이"와 함께해주세요.'] ['37'] ['11'] 120249
['따뜻함을 겸비한 멋스러움, FW21 프리미엄 수트 셋업 '] ['104'] ['19'] 121048
['[앵콜]2만원대로만나는 좋은재료에 깔끔한 컬러와 퀼러티 나만을 위한 안경'] ['44'] ['9'] 121407
['[한정수량] 지금보다 가을에 더 필요한, 유행안타는 365선글라스 일몬도'] ['98'] ['8'] 119050
['[자체 제작] 안팎으로 입기 좋은 순면 100% "티셔츠/맨투맨/후드티"'] ['36'] ['22'] 90319
['글로벌 감각의 Z세대 브랜드, TUO&MIO의 첫 컬렉션'] ['10'] ['3'] 120955
['통통이도 예쁘고 편한 옷을 입고 싶다. 789 가을 롱블라우스  바바리 '] ['0'] ['4'] 121293
['[3만원대] 디자인과 실용성 모두를 이 가방 하나로 가져가세요 '] ['6130'] ['1,394'] 118824
['[2.7억 펀딩] 원단빼고 풀카본! 세상에 없는 리얼 풀카본 장우산!'] ['14251'] ['2,105'] 114475
['[만족도 4.9 앵콜!] 블랙야크 트래블러DㅣBOA 다이얼 트레킹화'] ['5975'] ['964'] 11687

In [8]:
url = hrefs[1]
webpage = requests.get(url)
soup = bsp(webpage.content, "html.parser")

# ntitle
title = soup.find_all('a')
ntitle = title[4].contents
ntitle

#ncontent
reqheader = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0;Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'} 
src=requests.get(url,headers=reqheader)
cont=src.content.strip()#.decode('cp949') # 디코드 하고 가져오기
html=bsp(cont,'html.parser')
tgts=html.select('.inner-contents p')

ncontent_list = []
for tgt in tgts:
    ncontent_list.append(tgt.text)
ncontent = " ".join(ncontent_list)

#npercent
percent = soup.find_all('p', 'achievement-rate')
npercent = percent[0].contents[0].contents

#nsup
sup = soup.find_all('p', 'total-supporter')
nsup = sup[0].contents[0].contents

#nkey
nkey = url.split('/')[6]

#img
img_url = []
imgs = soup.find_all('img')
for img in imgs:
    url = img.get('data-src')
    
    if url != None:
        img_url.append(url)
    else:
        url = img.get('src')
        img_url.append(url)
len(img_url)

#print(ntitle)

#insert_wadiz(ntitle, ncontent, npercent, nsup, nkey)

70

In [20]:
img_url

['https://cdn.wadiz.kr/wwwwadiz/green001/2021/0630/20210630124925959_120950.jpg/wadiz/format/jpg/quality/80/optimize',
 'https://cdn.wadiz.kr/wwwwadiz/green001/2021/0630/20210630124928505_120950.jpg/wadiz/format/jpg/quality/80/optimize',
 'https://cdn.wadiz.kr/wwwwadiz/green001/2021/0630/20210630124931250_120950.jpg/wadiz/format/jpg/quality/80/optimize',
 'https://cdn.wadiz.kr/wwwwadiz/green001/2021/0630/20210630124933060_120950.jpg/wadiz/format/jpg/quality/80/optimize',
 'https://cdn.wadiz.kr/wwwwadiz/green001/2021/0630/20210630124934590_120950.jpg/wadiz/format/jpg/quality/80/optimize',
 'https://cdn.wadiz.kr/ft/images/green001/2021/0707/20210707155607245_41.jpg/wadiz/format/jpg/quality/80/optimize',
 'https://cdn.wadiz.kr/ft/images/green001/2021/0707/20210707155903247_69.jpg/wadiz/format/jpg/quality/80/optimize',
 'https://cdn.wadiz.kr/ft/images/green001/2021/0707/20210707160123407_71.jpg/wadiz/format/jpg/quality/80/optimize',
 'https://cdn.wadiz.kr/ft/images/green001/2021/0707/20210

In [22]:
img_url_set = set(img_url)
len(img_url_set)

70

In [23]:
img_for_txt = Image.open(request.urlopen(img_url[5]))

Print_img = pytesseract.image_to_string(img_for_txt, lang = 'kor')
Print_img

'하는\n\n기\n\n사람을\n\n \n\n|\n\n21\n\n|\n\n=\n\n니그 후\n\n \n\n7)\n1\n\n~\n\n=\n\n[=\n\n기\n\n[0\n\n의지\n\n \n\n  \n\n인의 발에\n\n|\n\n형과 한국\n\n더\n\x0c'

In [26]:
imgs

[<img alt="" class="img-responsive" src="https://cdn.wadiz.kr/wwwwadiz/green001/2021/0630/20210630124925959_120950.jpg/wadiz/format/jpg/quality/80/optimize"/>,
 <img alt="" class="img-responsive" src="https://cdn.wadiz.kr/wwwwadiz/green001/2021/0630/20210630124928505_120950.jpg/wadiz/format/jpg/quality/80/optimize"/>,
 <img alt="" class="img-responsive" src="https://cdn.wadiz.kr/wwwwadiz/green001/2021/0630/20210630124931250_120950.jpg/wadiz/format/jpg/quality/80/optimize"/>,
 <img alt="" class="img-responsive" src="https://cdn.wadiz.kr/wwwwadiz/green001/2021/0630/20210630124933060_120950.jpg/wadiz/format/jpg/quality/80/optimize"/>,
 <img alt="" class="img-responsive" src="https://cdn.wadiz.kr/wwwwadiz/green001/2021/0630/20210630124934590_120950.jpg/wadiz/format/jpg/quality/80/optimize"/>,
 <img class="fr-fil fr-dib" data-fr-image-pasted="true" data-height="2719" data-src="https://cdn.wadiz.kr/ft/images/green001/2021/0707/20210707155607245_41.jpg/wadiz/format/jpg/quality/80/optimize" da

In [47]:
comm = soup.find_all('span','count-total comment-total')
comm[0].text

'5'

In [40]:
sup

[<p class="total-supporter"><strong>562</strong>명의 서포터</p>,
 <p class="total-supporter"><strong>562</strong>명의 서포터</p>]

In [49]:
news = soup.find_all('span', 'count-total')
news[0].text

'2'

In [34]:
soup

<!DOCTYPE doctype html>

<html lang="ko">
<head>
<style>.async-hide { opacity: 0 !important} </style>
<script>(function(a,s,y,n,c,h,i,d,e){s.className+=' '+y;h.start=1*new Date;
h.end=i=function(){s.className=s.className.replace(RegExp(' ?'+y),'')};
(a[n]=a[n]||[]).hide=h;setTimeout(function(){i();h.end=null},c);h.timeout=c;
})(window,document.documentElement,'async-hide','dataLayer',4000,
{'GTM-NDMQ9N4':true});</script>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width,initial-scale=1,shrink-to-fit=no,user-scalable=no,viewport-fit=cover" name="viewport"/>
<meta content="가능성이 현실이 되는 테크부터 믿을 수 있는 생산자의 푸드까지. 내 방식대로 내 세상을 바꾸는 아주 특별한 이야기, 펀딩으로 시작하자!" name="description"/>
<link href="https://www.wadiz.kr/web/campaign/detail/120950" rel="canonical"/>
<meta content="190622721088710" property="fb:app_id"/>
<meta content="website" property="og:type"/>
<meta content="ko_KR" property="og:locale"/>
<meta content="와디즈" property="og:site